Import All necessary library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

First read all the final datasets for training and test data that have been generated.

In [3]:
train_data = pd.read_csv("worked_data_train.csv", index_col = 0)
Test = pd.read_csv("worked_data_test.csv", index_col = 0)

Impute object columns with mode

In [4]:
for feature in train_data.select_dtypes("object").columns:
    train_data[feature].fillna(train_data[feature].mode()[0], inplace = True)

Now, we impute the float and integer columns in train dataset with mean and median respectively

In [6]:
train_data = pd.get_dummies(train_data)
for feature in train_data.select_dtypes("float64").columns:
    train_data[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in train_data.select_dtypes("int64").columns:
    train_data[feature].fillna(train_data[feature].median(), inplace = True)

Do the same thing for test dataset

In [7]:
Test =  pd.get_dummies(Test)
for feature in Test.select_dtypes("float64").columns:
    Test[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in Test.select_dtypes("int64").columns:
    Test[feature].fillna(train_data[feature].median(), inplace = True)

Check if there are any columns that is not contained in test dataset except the Target column

In [10]:
def fun(y,x):
    for i in x:
        if i not in y:
            print(i)

In [11]:
fun(Test,train_data)

TARGET
CODE_GENDER_XNA
NAME_INCOME_TYPE_Maternity leave
NAME_FAMILY_STATUS_Unknown


Drop the unatched columns

In [12]:
train_data = train_data.drop(['CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown'], 1)

In [13]:
train_data.shape

(307511, 450)

For training our model, we use cross validation in order to perform better learning. For this purpose, we set the number of fold to 10. and by the end we get the foldsof the cross validation process.

In [14]:
def get_cross_val(dataset)
    k_fold = KFold(n_splits = 10, random_state = 42, shuffle = True)
    result = []
    for train_index, test_index in k_fold.split(dataset):
        to_append = {"Train" : dataset.iloc[train_index,], "Test" : dataset.iloc[test_index]}
        result.append(to_append)
    return result

# Run the function
folds = get_cross_val(train_data)

Now we are going to perform logistic regression and XgBoost and compare the result

## Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

For Logistic regression, we set the parameter C to 0.001, fit the model, get the probabilities, and finally check the ROC score for the model.

In [66]:
def logisticRegression(data):
    roc_scores = []
    for fold in folds:
        log_reg = LogisticRegression(C = 0.0001)
        log_reg.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
        probabilities = log_reg.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
        roc_scores.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    return print(sum(roc_scores) / len(roc_scores))

Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [67]:
logisticRegression(folds)

0.6166448087998891

Now we run the model for the test dataset, and get the csv of predicted probabilities

In [15]:
Test_or= pd.read_csv("application_test.csv")

In [69]:
def getResult (model, data):
    result = data['SK_ID_CURR']
    P = model.predict_proba(Test)
    preds = pd.DataFrame()
    preds['SK_ID_CURR'] = result
    l = []
    for i in P:
        l.append(i[1])
    preds['TARGET'] = l
    name = str(model) + '.csv'
    preds.to_csv(name, index = False)

#Run the program
getResult(log_res, Test_or)

Now, we try for XGBoost

## XGBoost

In [16]:
from xgboost import XGBClassifier

In [17]:
def XGBoost (data)
    roc_scores_xg = []
    for fold in data:
        xg = XGBClassifier()
        xg.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
        probabilities = xg.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
        roc_scores_xg.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    return print (sum(roc_scores_xg) / len(roc_scores_xg))

Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [18]:
XGBoost(folds)

0.7658449900783483

run the model for the test dataset, and get the csv of predicted probabilities

In [19]:
result_xg = Test_or['SK_ID_CURR']
P = xg.predict_proba(Test)
preds = pd.DataFrame()
preds['SK_ID_CURR'] = result_xg
l = []
for i in P:
    l.append(i[1])
preds['TARGET'] = l
preds.to_csv('xg2.csv', index = False)